In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
             return nothing
        else
             # segments parallel: no intersection
             return nothing
        end
    end
    return α,β
end

intersection (generic function with 1 method)

## 0) Benchmark vecchia funzione 


In [3]:
line1 = [0.0 0.0; 0.0 1.0]
line2 = [1.0 1.0; 0.0 1.0]
@btime intersection(line1, line2)

  884.265 ns (25 allocations: 592 bytes)


## 1) Controllo se la funzione è type unstable

In [1]:
@code_warntype intersection(line1, line2)

LoadError: UndefVarError: line1 not defined

la funzione è type unstable in quanto ho nell'output la stringa: 

    Body::Union{Nothing, Tuple{Any,Any}}
    
Il problema è che la funzione può ritornare una tupla di Float64 oppure nothing a seconda se entro nel then oppure nell'else. Il problema è stato risolto sostituendo nothing con ().

if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
             return ()
        else
             # segments parallel: no intersection
             return ()
        end